In [1]:
import numpy as np
import pandas as pd
import pylab as pl
import scipy.optimize as opt
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
# from google.colab import files
%matplotlib inline
from sklearn import svm
import pickle
from sklearn.metrics import classification_report, confusion_matrix, f1_score # jaccard_similarity_score
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [2]:
pest_data = pd.read_csv("BinaryPesticideData.csv")
pest_data

,Pesticide Name,LOD,EPA Tolerance (ppm),ExactMolWt,HeavyAtomMolWt,NumRadicalElectrons,NumValenceElectrons,HeavyAtomCount,NHOHCount,NOCount,...,NumAromaticRings,NumHAcceptors,NumHDonors,NumHeteroatoms,NumRotatableBonds,NumSaturatedCarbocycles,NumSaturatedHeterocycles,NumSaturatedRings,RingCount,Class
0,Desmedipham,0.0269,NT,300.111007,284.186,0,114,22,2,6,...,2,4,2,6,4,0,0,0,2,1
1,Thiacloprid,0.0004,NT,252.023645,243.658,0,82,16,0,4,...,1,4,0,6,2,0,1,1,2,1
2,Phenthoate,0.0060,NT,320.030588,303.236,0,106,19,0,4,...,1,6,0,7,7,0,0,0,1,1
3,Fenthion,0.0024,NT,278.020023,263.215,0,90,16,0,3,...,1,5,0,6,5,0,0,0,1,1
4,Dinotefuran,0.0100,NT,202.106590,188.102,0,80,14,2,7,...,0,4,2,7,3,0,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
222,Oxamyl,0.0080,T,219.067762,206.162,0,80,14,1,6,...,0,5,1,7,1,0,0,0,0,2
223,Carfentrazone ethyl,0.0030,T,411.036431,398.083,0,142,26,0,6,...,2,6,0,11,6,0,0,0,2,2
224,Endosulfan sulfate,0.0070,T,419.811796,416.880,0,114,20,0,4,...,0,4,0,11,0,1,1,2,3,2
225,Acetamiprid,0.0010,T,222.067224,211.591,0,78,15,0,4,...,1,3,0,5,2,0,0,0,1,2


In [3]:
pest_data.describe()
pest_data.drop(columns="NumRadicalElectrons", axis=1, inplace=True)
pest_data.describe()

,LOD,ExactMolWt,HeavyAtomMolWt,NumValenceElectrons,HeavyAtomCount,NHOHCount,NOCount,NumAliphaticCarbocycles,NumAliphaticHeterocycles,NumAliphaticRings,...,NumAromaticRings,NumHAcceptors,NumHDonors,NumHeteroatoms,NumRotatableBonds,NumSaturatedCarbocycles,NumSaturatedHeterocycles,NumSaturatedRings,RingCount,Class
count,227.000000,227.000000,227.000000,227.000000,227.000000,227.000000,227.000000,227.000000,227.000000,227.000000,...,227.000000,227.000000,227.000000,227.000000,227.000000,227.000000,227.000000,227.000000,227.000000,227.000000
mean,0.013992,314.329044,298.101727,110.634361,20.356828,0.722467,4.577093,0.215859,0.312775,0.528634,...,1.268722,4.281938,0.660793,6.590308,4.383260,0.145374,0.189427,0.334802,1.797357,1.713656
std,0.027355,141.878676,132.130233,56.013772,10.205000,1.033756,2.493710,0.596535,1.040747,1.399374,...,0.969919,2.494202,0.914197,2.954596,2.672091,0.452147,0.719066,0.932584,1.491389,0.453051
min,0.000100,44.026215,40.021000,18.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,0.002000,243.987613,228.380000,84.000000,15.000000,0.000000,3.000000,0.000000,0.000000,0.000000,...,0.000000,3.000000,0.000000,5.000000,2.000000,0.000000,0.000000,0.000000,1.000000,1.000000
50%,0.006000,302.012602,285.065000,106.000000,19.000000,0.000000,4.000000,0.000000,0.000000,0.000000,...,1.000000,4.000000,0.000000,6.000000,4.000000,0.000000,0.000000,0.000000,2.000000,2.000000
75%,0.015000,352.158152,332.715000,124.000000,23.000000,1.000000,5.500000,0.000000,0.000000,1.000000,...,2.000000,5.000000,1.000000,8.000000,6.000000,0.000000,0.000000,0.000000,2.000000,2.000000
max,0.250000,1730.968764,1589.017000,690.000000,123.000000,6.000000,28.000000,4.000000,12.000000,14.000000,...,3.000000,28.000000,6.000000,28.000000,15.000000,4.000000,8.000000,8.000000,14.000000,2.000000


In [4]:
pest_data.dtypes

Pesticide Name               object
LOD                         float64
EPA Tolerance (ppm)          object
ExactMolWt                  float64
HeavyAtomMolWt              float64
NumValenceElectrons           int64
HeavyAtomCount                int64
NHOHCount                     int64
NOCount                       int64
NumAliphaticCarbocycles       int64
NumAliphaticHeterocycles      int64
NumAliphaticRings             int64
NumAromaticCarbocycles        int64
NumAromaticHeterocycles       int64
NumAromaticRings              int64
NumHAcceptors                 int64
NumHDonors                    int64
NumHeteroatoms                int64
NumRotatableBonds             int64
NumSaturatedCarbocycles       int64
NumSaturatedHeterocycles      int64
NumSaturatedRings             int64
RingCount                     int64
Class                         int64
dtype: object

# *Data Split/Prep*

In [5]:
pest_feats = pest_data[['ExactMolWt', 'HeavyAtomMolWt', 'NumValenceElectrons', 'HeavyAtomCount', 'NHOHCount', 
                        'NOCount', 'NumAliphaticCarbocycles', 'NumAliphaticHeterocycles', 'NumAliphaticRings', 
                        'NumAromaticCarbocycles', 'NumAromaticHeterocycles', 'NumAromaticRings', 'NumHAcceptors', 
                        'NumHDonors', 'NumHeteroatoms', 'NumRotatableBonds', 'NumSaturatedCarbocycles', 
                        'NumSaturatedHeterocycles', 'NumSaturatedRings', 'RingCount']] # Removed LOD
pest_feats[0:20]
pest_feats.shape
pest_feats_array = pest_feats.values
pest_feats_array

array([[300.11100699, 284.186     , 114.        , ...,   0.        ,
          0.        ,   2.        ],
       [252.02364497, 243.658     ,  82.        , ...,   1.        ,
          1.        ,   2.        ],
       [320.03058765, 303.236     , 106.        , ...,   0.        ,
          0.        ,   1.        ],
       ...,
       [419.81179575, 416.88      , 114.        , ...,   1.        ,
          2.        ,   3.        ],
       [222.06722403, 211.591     ,  78.        , ...,   0.        ,
          0.        ,   1.        ],
       [201.03606822, 194.198     ,  68.        , ...,   0.        ,
          0.        ,   3.        ]])

In [6]:
from sklearn.preprocessing import StandardScaler

pest_feats_scaled = StandardScaler().fit_transform(pest_feats_array)  # Previously, a huge bias without scaling

# one feature had 99% explanation

pest_feats_pca = PCA(n_components=7).fit(pest_feats_scaled)
pest_feats_pca_values = pest_feats_pca.transform(pest_feats_scaled)
pest_feats_pca_values

array([[-0.06540902, -1.27481012, -0.82972949, ..., -1.54092359,
         0.77158139,  0.16882798],
       [-0.22807388,  0.83030492,  0.06828323, ...,  1.74832324,
        -1.87575783, -0.47576606],
       [-0.49175646, -0.28671791,  0.08667064, ..., -0.50544296,
         0.02085908, -0.08534201],
       ...,
       [ 2.81988053,  3.89969809,  1.59914926, ...,  0.86904859,
        -0.45699914,  2.12290421],
       [-1.92676775,  0.43405038, -0.04877979, ...,  1.59251151,
        -0.82708818, -0.1433168 ],
       [-1.84751632, -1.53616544,  0.60680837, ...,  2.50524897,
        -0.24091935, -0.7765139 ]])

In [7]:
pest_feats_pca.explained_variance_ratio_

array([0.49264112, 0.14572025, 0.12246059, 0.08576058, 0.05366027,
       0.03996117, 0.02622315])

In [8]:
pest_feats_pca.explained_variance_ratio_.sum()

0.9664271451575587

In [9]:
pest_data['Class'] = pest_data['Class'].astype(int)
y = np.asanyarray(pest_data['Class'])
y[0:5]

X = pest_feats[:]
X.shape

(227, 20)

In [10]:
X_train, X_test, y_train, y_test = train_test_split (pest_feats_pca_values, y, test_size=0.2, shuffle=True)
print ('X_train and y_train shape: ', X_train.shape, y_train.shape)
print ('X_test and y_test shape: ', X_test.shape, y_test.shape)

X_train and y_train shape:  (181, 7) (181,)
X_test and y_test shape:  (46, 7) (46,)


In [11]:
X_train = StandardScaler().fit_transform(X_train)
X_test = StandardScaler().fit_transform(X_test)

In [12]:
X_train

array([[ 0.37477983, -1.11007537,  0.60767688, ..., -0.94177047,
         0.33385039,  0.81723636],
       [-0.40789456, -1.0246495 ,  1.41328117, ...,  0.40911207,
        -0.75492685,  0.0615994 ],
       [ 0.88007708,  2.52837026,  1.17968016, ...,  0.71547884,
        -0.63319693,  2.60698494],
       ...,
       [-0.4382954 ,  0.75290964, -1.24152244, ..., -0.07999418,
         0.06126327,  0.05077998],
       [ 0.34648366, -1.12629665,  0.30084301, ...,  1.28418383,
         0.3271502 ,  1.04188408],
       [-0.58963635,  0.27166842, -0.07738258, ...,  1.75658686,
        -0.93362058,  0.47301213]])

In [13]:
X_test[0:5]

array([[-0.383252  ,  0.31048189, -0.24089188,  2.14634075, -0.82707235,
         0.51350751, -2.14505445],
       [ 0.12326276,  0.04128644, -0.46887092, -0.54675242, -0.67770352,
        -1.19477018,  0.83593574],
       [-0.49519382,  0.02157642, -0.4655678 , -0.46848736, -0.8310168 ,
        -0.20551645,  0.81170686],
       [-0.18025304, -0.89257922,  0.56174611,  0.31107016,  0.72345197,
        -0.03110676,  0.64215115],
       [ 0.00329881, -1.58238513,  1.37816786,  0.27717164, -1.82501254,
         0.42933701, -0.27570067]])

In [14]:
y_train

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 2, 2, 1, 2, 1, 2, 2, 2, 2,
       2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 1, 2, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2,
       2, 1, 1, 2, 2, 1, 1, 2, 2, 2, 2, 1, 1, 2, 2, 2, 1, 1, 2, 1, 2, 2,
       1, 2, 1, 1, 2, 2, 2, 2, 1, 2, 2, 1, 1, 1, 1, 1, 2, 2, 1, 2, 1, 2,
       2, 1, 2, 2, 2, 1, 2, 2, 1, 2, 2, 2, 1, 2, 1, 1, 1, 2, 2, 1, 2, 2,
       1, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 1,
       2, 2, 2, 2, 1, 2, 2, 2, 1, 1, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 1,
       2, 2, 1, 1, 2, 2, 2, 1, 2, 2, 2, 2, 1, 2, 1, 2, 2, 2, 2, 2, 1, 2,
       2, 2, 2, 2, 2])

In [15]:
y_test

array([2, 2, 2, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2,
       2, 2, 2, 1, 2, 2, 1, 1, 1, 1, 1, 2, 2, 2, 2, 1, 2, 1, 2, 1, 2, 2,
       2, 2])

# SVM Model Dev

In [16]:
clf = svm.SVC(kernel='rbf', class_weight="balanced")
clf.fit(X_train, y_train.ravel())

SVC(class_weight='balanced')

In [17]:
# import pickle
# s = pickle.dumps(clf)

In [18]:
y_hat_svm = clf.predict(X_test)
y_hat_svm

array([2, 2, 1, 2, 2, 1, 1, 2, 2, 1, 1, 2, 1, 2, 1, 1, 2, 1, 2, 1, 2, 2,
       1, 2, 2, 1, 2, 1, 2, 1, 2, 1, 1, 2, 2, 2, 2, 2, 1, 1, 2, 2, 2, 1,
       1, 1])

## Model Eval

In [19]:
print (classification_report(y_test, y_hat_svm))

              precision    recall  f1-score   support

           1       0.29      0.50      0.36        12
           2       0.76      0.56      0.64        34

    accuracy                           0.54        46
   macro avg       0.52      0.53      0.50        46
weighted avg       0.64      0.54      0.57        46



In [20]:
f1_score(y_test, y_hat_svm, average='weighted')

0.5709117706170028

In [21]:
jaccard_similarity_score(y_test, y_hat_svm)

NameError: name 'jaccard_similarity_score' is not defined

In [ ]:
confusion_matrix(y_test, y_hat_svm, labels=[1,2])

# Decision Tree


In [ ]:
pestTree = DecisionTreeClassifier(criterion="entropy", splitter='best', max_depth = 10)
pestTree.fit(X_train, y_train)

In [ ]:
predTree = pestTree.predict(X_test)
predTree[0:5]

In [ ]:
print("DecisionTrees's Accuracy: ", metrics.accuracy_score(y_test, predTree))

In [ ]:
pestTree.tree_.max_depth

In [ ]:
print (classification_report(y_test, predTree))

In [ ]:
#for i in 50: 
#  feat_arr = pestTree.feature_importances_

In [ ]:
pestTree.feature_importances_

In [ ]:
pest_feats.columns

In [ ]:
#from sklearn.externals.six import StringIO
# import pydotplus # provides a Python Interface to the Graphviz Dot language.
# import matplotlib.image as mpimg
# from sklearn import tree
# %matplotlib inline 

# dot_data = StringIO()
# filename = "pesttree.png"
# featureNames = pest_feats.columns[0:22]
# targetNames = pest_data["Class"].unique().tolist()

# out=tree.export_graphviz(pestTree,feature_names=featureNames, out_file=dot_data, class_names= np.unique(y_train), filled=True,  special_characters=True,rotate=False)  
# graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
# graph.write_png(filename)

# img = mpimg.imread(filename)
# plt.figure(figsize=(100, 200))
# plt.imshow(img,interpolation='nearest')

# Logistic Regression

In [ ]:
LR = LogisticRegression(C=100, solver='saga').fit(X_train, y_train)
y_hat_lr = LR.predict(X_test)
y_hat_lr[0:10]

In [ ]:
y_hat_prob = LR.predict_proba(X_test)
y_hat_prob[0:10]

In [ ]:
print (classification_report(y_test, y_hat_lr))